# **Sitio Web Dinámico**: la info va a un servidor y de forma iterativa llena la pagina

un sitio estático se hace a mano 

In [12]:
import requests
from bs4 import BeautifulSoup

In [21]:
main_url = 'https://books.toscrape.com'
books_file = './books.csv'
headers = {'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux; x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}

In [22]:
pages_to_scrape = 10

In [39]:
def get_all_info_from_page(url:str):
    
    html = requests.get(url, headers  = headers)
    bs = BeautifulSoup(html.content, 'html.parser')
    
    # find all regresa una lista con todo lo que pido de etiquetas
    links = bs.find_all('img',
                       class_ = 'thumbnail' # como class es reservada de python para scape a la palabra 
                                            # es con "_" despues de la palabra reservada
                       )
    # con lo anterior obtengo una lista de todas las imagenes de la página
    
    # me fijo en todos los hijos de los links
    for link in links: # itero sobre todas las imagenenes de la página
        print(link.parent['href'])
        html = requests.get(main_url + '/catalogue/' + link.parent['href'], #el papa es el que me da la referencia de la pagina del libro
                           headers = headers                # de manera especifica 
                           )
        
        # lo anterior me mete a la pagina de cada libro donde voy a empezar a sacar las
        # características del libro
        
        # con esta sopa lo que haces es simular dar un nuevo clic
        # es decir este bs está buscando dentro de cada libro 
        bs = BeautifulSoup(html.content,
                           'html.parser'
                          )
        ################################## titulo
        # me traigo el titulo del libro a diferencia del fins_all 
        # el fins solo trae un objeto
        title = bs.find('h1').text
        
        ################################### precio
        price = bs.find('p',
                        class_ = 'price_color' # me traigo exactamente el precio porque está aqui
                       ).text
        
        ################################### stock disponible
        stock = bs.find('p',
                        class_ = 'availability').text.replace('\n', '').lstrip()
        
        ###################################descripcion
        #como la descripción es un hermano nos permite llegar a él
        # por medio del primer hermano 
        try:    
            description = bs.find('div',
                                  id = 'product_description'
                                 ).next_sibling.next_sinling.text # hay doble salto de hermano porque el primero es un salto de linea

        except:
            description = 'no tinene'
            
        
        ##################################### me traigo el upc (es como un identificador )
        upc = bs.table.tr.td.text #dentro de la sopa, entro a la tabla, luego al tr, y al final llego al td (upc)
        
        #################################### tipo de producto      
        product_type = bs.table.tr.next_sibling.next_sibling.td.text
        
        #guardar la información
        with open(books_file, 'a') as file_object:
            file_object.write(f'{title}, {price}, {stock}, {description}, {upc}, {product_type}\n ')

In [41]:
page_numbers = range(1, pages_to_scrape + 1)
for page_number in page_numbers:
    print(str(page_number))
    get_all_info_from_page(main_url + '/catalogue/' + 'page-'  + str(page_number) + '.html')

1
a-light-in-the-attic_1000/index.html
tipping-the-velvet_999/index.html
soumission_998/index.html
sharp-objects_997/index.html
sapiens-a-brief-history-of-humankind_996/index.html
the-requiem-red_995/index.html
the-dirty-little-secrets-of-getting-your-dream-job_994/index.html
the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html
the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html
the-black-maria_991/index.html
starving-hearts-triangular-trade-trilogy-1_990/index.html
shakespeares-sonnets_989/index.html
set-me-free_988/index.html
scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html
rip-it-up-and-start-again_986/index.html
our-band-could-be-your-life-scenes-from-the-american-indie-underground-1981-1991_985/index.html
olio_984/index.html
mesaerion-the-best-science-fiction-stories-1800-1849_983/index.html
libertarianism-for-beginners_982/index.html
its-only-the-himalayas_98

In [42]:
import scrapy